In [3]:
# TODO Currently accesses .pos files from local directory. Must change to download from Zenodo when available

# Zachary Katz
# zachary_katz@mines.edu
# August 2024

# Make and save the catalog of Whillans Stick Slip Events from .pos files

# Imports
import os

import src.Catalog.CatalogWrapper as CatWrap
import src.Catalog.Catalog as Catalog
import logging

################################################################################
########################## User Defined Variables ##############################
# fmt: off
dir = "/mnt/d/csrs_2024/all" 
stas = ["la01", "la02", "la03", "la04", "la05", "la06", "la07", "la08", "la09",
        "la10", "la11", "la12", "la13", "la14", "la15", "la16", "la17", "la18",
        "ws04", "ws05",
        "gz01", "gz02", "gz03", "gz04", "gz05", "gz06", "gz07", "gz08", "gz09",
        "gz10", "gz11", "gz12", "gz13", "gz14", "gz15", "gz16", "gz17", "gz18",
        "gz19", "gz20"
        "mg01", "mg02", "mg03", "mg04", "mg05", "mg06", "mg07",
        "slw1"
        ]
stas = ["la14","la15"]
year_arr = [["2010", "2011"]]
year_arr = [["2014", "2015"]]
max_gap_len = 120

# Currently implemented: ERROR, WARNING, INFO
# Each level also includes all levels above it.
loglevel = "INFO" 
# fmt: on
########################## User Defined Variables ##############################
################################################################################

In [4]:
def set_log_level(loglevel: str) -> None:
    """Set logging level for the run

    Parameters
    ----------
    loglevel : str
        Logging level
    """
    FORMAT = "%(asctime)s %(name)s %(levelname)s: %(message)s"
    DATEFMT = "%Y-%m-%d %H:%M:%S"
    if loglevel == "ERROR":
        logging.basicConfig(
            format=FORMAT, datefmt=DATEFMT, level=logging.ERROR, force=True
        )
    elif loglevel == "WARNING":
        logging.basicConfig(
            format=FORMAT, datefmt=DATEFMT, level=logging.WARNING, force=True
        )
    elif loglevel == "INFO":
        logging.basicConfig(
            format=FORMAT, datefmt=DATEFMT, level=logging.INFO, force=True
        )

In [5]:
# Setup logger
set_log_level(loglevel)
logger = logging.getLogger(__name__)

# Create Catalog
for years in year_arr:
    cats = []
    for sta in stas:
        interpolation_time, run = Catalog.set_interpolation_time(sta, years)
        if run:
            logger.info(f"Creating Dataframe for {sta}")
            cat = Catalog.Datastream(
                os.path.join(dir, sta), sta, years, interpolation_time
            )
            logger.info(f"Interpolating {sta}")
            cat.findgaps(interpolation_time)
            cats.append(cat)

    logger.info("Creating Catalog")
    catalog = CatWrap.full_catalog_run(
        cats, years, window=600, slide=25, active_stas=2, cull_time=30, plot=False
    )

2024-08-23 09:20:37 __main__ INFO: Creating Dataframe for la14
2024-08-23 09:20:50 __main__ INFO: Interpolating la14
2024-08-23 09:20:59 __main__ INFO: Creating Catalog
2024-08-23 09:20:59 src.Catalog.Catalog INFO: Linear Least Squares on la14
2024-08-23 09:21:20 src.Catalog.Catalog INFO: Picking Events
2024-08-23 09:21:20 src.Catalog.Catalog INFO: Average: 0.8245161699320017, Standard Deviation: 1.7539553726775758


In [6]:
# Plot Example Events
for event in catalog[:10]:
    Catalog.plot_event(event)